# Import libraries and init

In [1]:
# import ROOT -- don't need this for now
import numpy as np
import pandas as pd
from collections import OrderedDict
import math

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
plt.style.use('default')
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams.update({'font.size': 9})

import uproot # uproot needs to be last for some reason

In [2]:
import sys
sys.version
sys.version_info

sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)

In [3]:
from functools import reduce
import operator

# Load ntuples from analyzer

Define parameters in separate block so can change them without loading files all over again

In [10]:
background = ['NoBPTX']
cuts = np.arange(0,6)
branchPath = 'SREffi_dsa'
baseDir = '~/nobackup/signal_region/2018/backgrounds/'
#fileName = lambda Mchi, dMchi, ctau: baseDir + f'Mchi-{Mchi}_dMchi-{dMchi}_ctau-{ctau}.root'

#tres input
#fileName = 'root://cmsxrootd.fnal.gov//store/group/lpcmetx/iDM/Ntuples/2018/backgrounds/WWJJToLNuLNu/WWJJToLNuLNu_EWK_13TeV-madgraph-pythia8/WWJJToLNuLNu/190309_053107/0000/output_1.root'
#fileName = baseDir+'WWTo2L2Nu/output_1-5.root'
#fileName = 'root://cmsxrootd.fnal.gov//store/group/lpcmetx/iDM/Ntuples/2018/backgrounds/WWTo2L2Nu/WWTo2L2Nu_13TeV-powheg/WWTo2L2Nu_13TeV-powheg/190308_155346/0000/output_1-5.root'
fileName = 'root://cmsxrootd.fnal.gov//store/group/lpcmetx/iDM/Ntuples/2018/backgrounds/NoBPTX/crab_noBPTX_v3/190612_154618/0000/output_1-2.root'
numParams = len(background)
countMasses = 0
    

#masses = [('60p0','20p0'),('6p0','2p0'),('52p5','5p0'),('5p25','0p5')]
masses = [('60p0','20p0')]
printMasses = lambda mass: f'({float(mass[0].replace("p","."))-float(mass[1].replace("p","."))/2}, ' + \
                                f'{float(mass[0].replace("p","."))+float(mass[1].replace("p","."))/2}) GeV'
mchis = dict([(mass[0], printMasses(mass)) for mass in masses])
ctaus = [1]#,10, 100]#, 1000]

numCuts = np.arange(0,6)
labels = [ f'cut{cut}' for cut in numCuts ]
cutDescriptions = ['cut1: MET/MHT trigger fired (120 GeV)', \
                   'cut2: j1 pT > 120 GeV, <= 2j w/ pT > 30 GeV', \
                   'cut3: mu1 pT > 5 GeV, 0.1 < |dxy| < 700 cm', \
                   'cut4: mu2 pT > 5 GeV, 0.1 < |dxy| < 700 cm', \
                   r'cut5: $|\Delta\Phi$(MET, mu pair)| < 0.4'
                  ]

branchPath = 'SREffi_dsa'
#baseDir = '/uscms/home/mreid/nobackup/signal_region/2018/pileup/'
#baseDir = '../iDM_analysis_skimming/washAOD/'
#baseDir = '/uscms/home/mreid/../../a/as2872/nobackup/iDM/AndreAnalysis/CMSSW_10_4_0_pre2/src/Firefighter/washAOD/SROptimization/'
#baseDir = '/uscms/home/mreid/nobackup/signal_region/2018/GenFilter_1or2Jets_longname/'
#fileName = lambda Mchi, dMchi, ctau: baseDir + f'pileup_trackquality_Mchi-{Mchi}_dMchi-{dMchi}_ctau-{ctau}.root'

Load signal files

In [16]:
numParams = len(masses)*len(ctaus)
countParam = 1

trees = OrderedDict({})
treesglobal = OrderedDict({})
genInfo = OrderedDict({})
numEvents = OrderedDict({})
for (Mchi, dMchi) in masses:
#        genInfo[Mchi][ctau] = uproot.open(fileName(Mchi, dMchi, ctau))['GEN/gen']#.pandas.df(flatten=False)
    trees[Mchi] = uproot.open(fileName)[branchPath + f'/cutsTree']#.pandas.df(flatten=False)
    treesglobal[Mchi] = uproot.open(fileName)['SREffi_gbm' + f'/cutsTree']#.pandas.df(flatten=False)
    print(f'{countParam} of {numParams}: ' + fileName)
    countParam += 1

1 of 1: root://cmsxrootd.fnal.gov//store/group/lpcmetx/iDM/Ntuples/2018/backgrounds/NoBPTX/crab_noBPTX_v3/190612_154618/0000/output_1-2.root


Create pandas dataframes for different physics objects now that uproot won't flatten them anymore
(does make code faster)

In [17]:
leadingJet = OrderedDict({}); MET = OrderedDict({});
muons = OrderedDict({}); vertex = OrderedDict({});
cuts = OrderedDict({}); cutsCrit = OrderedDict({});
checkmuons = OrderedDict({}); gen = OrderedDict({})
checkgmuons = OrderedDict({}); beamhalo = OrderedDict({});
for mchi in mchis:
    leadingJet[mchi] = trees[mchi].pandas.df(['recoPFJetPt','recoPFJetEta','recoPFJetPhi']).loc[(slice(None),0),slice(None)].reset_index(level=1)
    MET[mchi] = trees[mchi].pandas.df(['recoPFMetPt', 'recoPFMetPhi'])#.reset_index(level=1)
    muons[mchi] = trees[mchi].pandas.df(['recoPt','recoEta','recoPhi','recoDxy','recoDz']).reset_index(level=1)
    vertex[mchi] = trees[mchi].pandas.df(['recoDr','recoVxy','recoVz']).reset_index(level=1)
    cuts[mchi]= trees[mchi].pandas.df('cutsVec*')
    cutsCrit[mchi] = [ cuts[mchi][f'cutsVec[{cut}]'] == 1 for cut in numCuts ]
#        checkmuons[mchi][ctau] = trees[mchi][ctau].pandas.df(['recoPt','recoEta','recoPhi','recoDxy','recoDz','trackHits','trackPlanes','trackChi2','qualityTrack','isGenMatched','GenDR']).reset_index(level=1)
#        checkgmuons[mchi][ctau] = treesglobal[mchi][ctau].pandas.df(['recoPt','recoEta','recoPhi','recoDxy','recoDz','trackHits','trackPlanes','trackChi2','qualityTrack','isGenMatched','GenDR']).reset_index(level=1)
#        beamhalo[mchi][ctau] = treesglobal[mchi][ctau].pandas.df(['fired','beamHaloHcal','beamHaloEcal','beamHaloCSC','beamHaloGlobal','beamHaloGlobalsuper','beamHaloCSCtight','trackHits','trackPlanes','trackChi2','qualityTrack']).reset_index(level=1)

Separately calculate the phi average angle between the two muons -- for use in DeltaPhi(MET, muon pair) -- since it takes a while

In [26]:
for cut in np.arange(0,6):
    cutsToApply = cutsCrit['60p0'][cut]
    cutFlowDict[back].append(len(dfs[back][cutsToApply]))
        
cutFlowDf = pd.DataFrame.from_dict(cutFlowDict)
cutFlowDf

NameError: name 'dfs' is not defined

In [ ]:
# Helper function to calculate average angles
# This takes a few seconds to run, since we 
# are using the apply method
def calcAvgAngle(group):
    x = np.cos(group['recoPhi'].iloc[0]) + np.cos(group['recoPhi'].iloc[-1])
    y = np.sin(group['recoPhi'].iloc[0]) + np.sin(group['recoPhi'].iloc[-1])
    return math.atan2(y/2, x/2)
avgMuonAngle = OrderedDict({})
for mchi in mchis:
    avgMuonAngle[mchi] = muons[mchi].groupby('entry').apply(calcAvgAngle)

# Plot everything together

To plot individual plots separately, copy relevant block of code and replace "axes[i,j]" with "plt", and take note that some of the functions change name, e.g. axes[i,j].set_xlabel() --> plt.xlabel() and axes[i,j].set_ylim() --> plt.ylim().

The other option is to copy the block and add "fig, ax = plt.subplots(1,1)" at the top, and then do the replacement axes[i,j] --> ax elsewhere.

## Helper functions

In [14]:
insets = True
log = False
histtype ='step'
iheight="45%"
iwidth="35%"

def plot_inset(data, axis, kwargs={}):
    #if 'bins' not in kwargs: kwargs['bins'] = 10
    if 'histtype' not in kwargs: kwargs['histtype'] = histtype#build
    if insets:
        axins = inset_axes(axis, width=iwidth, height=iheight)
        for (cut,datum) in enumerate(data):
            ret = axins.hist(datum, **kwargs)
            if cut == 1:
                axins.set_ylim(100,1.1*max(ret[0]))
        return axins

def plot_full(data, axis, kwargs={}, labels = []):
    if 'bins' not in kwargs: kwargs['bins'] = 50
    if 'histtype' not in kwargs: kwargs['histtype'] = histtype
    if 'log' not in kwargs: kwargs['log'] = log
    for (cut,datum) in enumerate(data):
        ret = axis.hist(datum, label=(labels[cut] if len(labels) > 0 else ''), **kwargs)
        if cut == 0:
            axis.set_ylim(1, 1.5*max(ret[0]))
            
# Helper function to normalize angle differences to [-Pi, Pi]
# cond: if abs(phidiff) > Pi => phidiff = phidiff - 2*Pi*sign(phidiff)
def reducephi(row):
    if abs(row) > math.pi:
        return row - 2*math.pi*(row/abs(row))
    return row

## Plot MET and jet variables

# Scratch tests and comparisons

Comparing results before and after fixing Dxy acceptance (i.e. Dxy is a signed distance and can be < 0)